In [ ]:
import geopandas as gpd
from shapely.geometry import LineString, Point
import os

def generate_points_for_folder(input_folder, output_folder, method):
    # List all shapefiles in the input folder
    shapefiles = [f for f in os.listdir(input_folder) if f.endswith('.shp')]

    # Iterate over each shapefile
    for shapefile in shapefiles:
        # Form the full path to the shapefile
        input_shapefile = os.path.join(input_folder, shapefile)

        # Generate points for the current shapefile
        generate_points_for_shapefile(input_shapefile, output_folder, method)

def generate_points_for_shapefile(input_shapefile, output_folder, method):
    # Read the shapefile
    gdf = gpd.read_file(input_shapefile)

    # Get the coordinate system from the input lines
    crs = gdf.crs

    # Define intervals based on the method
    if method == 'MASW':
        interval = 5
    elif method == 'TDEM':
        interval = 50
    elif method == 'DMASW':
        interval = 9
    elif method == 'HVSR':
        interval = 100
    else:
        raise ValueError("Invalid method. Supported methods are 'MASW', 'TDEM', 'DMASW', and 'HVSR'.")

    # Create a list to store Point geometries and LineNames
    points_geom = []
    line_names = []

    # Iterate over each feature in the GeoDataFrame
    for idx, row in gdf.iterrows():
        line = row.geometry  # Assuming the default geometry column is used

        # Get the LineName for the current line
        line_name = row['Name']

        # Generate points along the line with the specified interval
        for point_id, distance in enumerate(range(0, int(line.length) + interval, interval), start=1):
            point = line.interpolate(min(distance, line.length))
            points_geom.append(Point(point.x, point.y))
            line_names.append(line_name)

    # Create a GeoDataFrame from the list of Point geometries
    points = gpd.GeoDataFrame(geometry=points_geom, crs=crs)

    # Add the required attribute columns
    points['LineName'] = line_names
    points['ID'] = points.groupby('LineName').cumcount() + 1
    points['Name'] = [f"{line_name}-{point_id}" for line_name, point_id in zip(line_names, points['ID'])]
    points['Type'] = method

    # Add X, Y, Latitude, and Longitude fields
    points['X'] = points.geometry.x
    points['Y'] = points.geometry.y
    points['Latitude'] = points.geometry.y  # Assuming Y corresponds to Latitude in WGS84
    points['Longitude'] = points.geometry.x  # Assuming X corresponds to Longitude in WGS84

    # Save the resulting points to a new shapefile with the method name
    output_shapefile = os.path.join(output_folder, f"{method}_{os.path.splitext(os.path.basename(input_shapefile))[0]}.shp")
    points.to_file(output_shapefile, driver='ESRI Shapefile')

    # Display success message
    print(f"Successful! The '{method}' points for '{input_shapefile}' have been created and saved to '{output_shapefile}'.")

# Example usage
input_folder = input("Enter the path to the input folder containing shapefiles: ")
output_folder = input("Enter the path to the output folder: ")
method = input("Enter the method ('MASW', 'TDEM', 'DMASW', or 'HVSR'): ").upper()

generate_points_for_folder(input_folder, output_folder, method)
